
## 第二次小作业内容  

**1. 作业内容：**  

* 使用多个自变量(两个或两个以上，自行选择)对恋爱情况(因变量)建立logistic回归模型  

**2. 具体要求：**  
    重复课上的workflow，包括:  
		
    1. 模型定义(要求附上基本的文字解释，格式参考附后）  
    
    2. MCMC采样，模型诊断图，后验参数解释  
        * 使用az.plot_trace绘制模型诊断图  
        * 对后验参数结果在模型中的意义进行文字解释  
        * 绘制后验回归模型(使用az.plot_hdi)  

    3. 使用定义好的模型，对新站点的结果进行预测并评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    4. 对本数据集的预测结果进行评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    5. 与课上的模型(log_model1:自变量为回避倾向; log_model2:自变量为性别)进行模型比较  

**3. 作业截止时间：12.19**  

**4. 作业提交于和鲸平台**  


* 注1：所有的文字解释仅需增加在notebook中，无需提交额外的文档  

* 注2：上述要求请使用pymc语法实现  

* 注3：在数据预处理时注意缺失值  

* 注4：每个小组内成员所使用的站点数据是相同的，但该作业为个人独立提交，小组内可以互相讨论  

| 组别  | 建立模型使用的站点 | 对新数据进行预测时使用的站点  |  
|---------|------------------------------|-----------------------|  
| 1 | METU   | UCSB  |  
| 2 | Oxford | Poland |  
| 3 | Serbia | VCU |  
| 4 | VCU |Oxford |  
| 5 | UCSB |Serbia |  


**模型定义文字描述格式参考：**  

1. 自变量：xx  

2. 因变量：xx  

3. 数据关系：  

$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1 & {\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}}}{1 + e^{\beta_0 + \beta_1 X_{i1}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N()  \\  
               & & \beta_1  & \sim N(). \\  
\end{array}  
$$  

### 1.模型定义

In [1]:
# 数据包以及数据库导入
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

In [2]:
#对数据进行初步处理
#通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')

# 选取牛津站点的数据
df = df_raw[df_raw["Site"] == "Oxford"]

# 选取变量romantic、anxiety、avoidance
df = df[["romantic", "anxiety_r", "avoidance_r","sex"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df["sex"] =  np.where(df['sex'] == 1, 0, 1)

#设置索引
df["index"] = range(len(df))
df = df.set_index("index")

In [3]:
#模型构建
with pm.Model() as log_model1:
    # 此处对coords的定义方式进行了更改，因为后续我们需要进行对新数据的预测
    # 因此将维度定义成可更改的
    log_model1.add_coord('obs_id',df.index, mutable=True)
    anxiety = pm.MutableData("anxiety", df.anxiety_r, dims="obs_id")
    avoidance = pm.MutableData("avoidance", df.avoidance_r, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety + beta_2 * avoidance, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [4]:
# 可视化模型（贝叶斯变量因果图）
# 我们使用 pymc 自带的 `model_to_graphviz` 方法来可视化模型中各变量的因果关系。
pm.model_to_graphviz(log_model1)

自变量：anxiety avoidance  
因变量：romantic  
数据关系：$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1 ,\beta_2& \stackrel{ind}{\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}+\beta2X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1}+\beta2X_{i2}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N\left(0, 0.5^2 \right)  \\  
               & & \beta_1  & \sim N\left(0, 0.5^2 \right)\\  
							 & & \beta_2  & \sim N\left(0, 0.5^2 \right)\\  
\end{array}  
$$

### 2. MCMC采样，模型诊断图，后验参数解释

In [5]:
#先验预测检验
log1_prior = pm.sample_prior_predictive(samples=50, 
                                          model=log_model1,
                                          random_seed=84735)

Sampling: [beta_0, beta_1, beta_2, y_est]


In [33]:
log1_prior

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [6]:
#mcmc采样
with log_model1:
    # MCMC 近似后验分布
    log_model1_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 8 seconds.


In [7]:
#模型诊断图
az.plot_trace(log_model1_trace,
              var_names=["beta_0","beta_1", "beta_2"],
              figsize=(20,15),
              compact=False)
plt.show()

<Figure size 2000x1500 with 6 Axes>

In [32]:
az.summary(log_model1_trace, var_names=["beta_0","beta_1", "beta_2"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.020,0.172,-0.345,0.301,0.001,0.001,31343.0,16062.0,1.0
beta_1,-0.694,0.186,-1.044,-0.347,0.001,0.001,28531.0,14551.0,1.0
beta_2,0.066,0.173,-0.250,0.398,0.001,0.001,30120.0,15611.0,1.0


In [8]:
# 通过 np.exp 将 beta 参数进行转换
az.plot_posterior(log_model1_trace, var_names=["beta_0","beta_1", "beta_2"], transform = np.exp)
plt.show()

<Figure size 2208x552 with 3 Axes>

### 后验参数解释  

以下的结果显示：  
- $\beta_0 = -0.02$，那么 $e^{\beta_0} ≈ 1$， 表明 X1、X2 为 0时，个体恋爱的可能性接近1。  
- $\beta_1 = -0.694$， $e^{\beta_0} = 0.51$， 表明焦虑分数每增加1个单位，个体恋爱的发生比变为之前的0.51倍。  
- $\beta_2 = 0.066$， $e^{\beta_0} = 1.1$， 表明回避分数每增加1个单位，个体恋爱的发生比变为之前的1.1倍  
- 然而，$\beta_0$  $\beta_2$的94%HDI包括0，说明回避分数不能有效预测恋爱发生的概率。 

In [9]:
#后验回归模型
with log_model1:
    log_model1_ppc = pm.sample_posterior_predictive(log_model1_trace, random_seed=84735) 

Sampling: [y_est]


In [10]:
az.summary(log_model1_trace, kind = "stats")

,mean,sd,hdi_3%,hdi_97%
beta_0,-0.020,0.172,-0.345,0.301
beta_1,-0.694,0.186,-1.044,-0.347
beta_2,0.066,0.173,-0.250,0.398
mu[0],1.402,0.419,0.607,2.186
mu[1],0.316,0.342,-0.326,0.958
...,...,...,...,...
pi[132],0.426,0.054,0.324,0.525
pi[133],0.547,0.053,0.448,0.649
pi[134],0.756,0.064,0.632,0.872
pi[135],0.339,0.056,0.232,0.444


In [11]:
#画出每个自变量对应的恋爱概率94%hdi值
fig, axes = plt.subplots(figsize=(18, 8))

az.plot_hdi(
    df.anxiety_r,
    log_model1_trace.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C2"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean = log_model1_trace.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = df.anxiety_r, 
             y= post_mean, 
             label="posterior mean", 
             color="C2"
             )
#绘制真实数据散点图
sns.scatterplot(x = df.avoidance_r, 
                y= df.romantic,label="observed data", 
                color='#C00000', 
                alpha=0.5)
#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

#图2
az.plot_hdi(
    df.avoidance_r,
    log_model1_trace.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="C1"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean = log_model1_trace.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = df.avoidance_r, 
             y= post_mean, 
             label="posterior mean", 
             color="C1"
             )
#绘制真实数据散点图
sns.scatterplot(x = df.avoidance_r, 
                y= df.romantic,label="observed data", 
                color='#C00000', 
                alpha=0.5
                )
#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

<Figure size 1800x800 with 1 Axes>

###     4. 对本数据集的预测结果进行评估  

In [12]:
log_model1_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw"))

<xarray.DataArray 'y_est' (obs_id: 136, sample: 20000)>
array([[1, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0]])
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 6 7 ... 128 129 130 131 132 133 134 135
  * sample   (sample) object MultiIndex
  * chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 0 1 2 3 4 5 6 ... 4993 4994 4995 4996 4997 4998 4999

In [12]:
#stack(sample = ("chain", "draw")：将每一个X对应的4*5000个后验预测值合并到一个维度sample
#对于每一个X，需要计算其20000个值的平均值，因此将dim设置为sample
pred_pi = log_model1_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
# 转换为数据框
pred_pi = pred_pi.to_dataframe()

In [13]:
#将原数据中的X 和Y存入数据框
pred_pi["anxiety"] = log_model1_ppc.constant_data.anxiety.values
pred_pi["avoidance"] = log_model1_ppc.constant_data.avoidance.values
pred_pi["romantic"] = log_model1_ppc.observed_data.y_est.values

#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,anxiety,avoidance,romantic,romantic_pred
obs_id,,,,,
0,0.79735,-1.975260,0.781027,1,1
1,0.57810,-0.323469,1.681138,1,1
2,0.69795,-1.172962,0.893541,1,1
3,0.43165,0.384442,0.218458,1,0
4,0.45320,0.148472,-1.244223,0,0
...,...,...,...,...,...
132,0.42590,0.337248,-0.737910,0,0
133,0.54525,-0.370663,-0.681653,1,1
134,0.75775,-1.644902,0.612256,1,1


In [15]:
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,45,24
1,26,42


In [16]:
# 计算a b c d的数量
true_positive = 0
false_positive = 0
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
# 代入公式
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.36619718309859156
敏感性: 0.0
特异性: 1.0


### 3. 使用定义好的模型，对新站点的结果进行预测并评估

In [17]:
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')

# 选取牛津站点的数据
df_new = df_raw[df_raw["Site"] == "Poland"]

# 选取变量romantic、anxiety、avoidance
df_new = df_new[["romantic", "anxiety_r", "avoidance_r","sex"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df_new["romantic"] =  np.where(df_new['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df_new["sex"] =  np.where(df_new['sex'] == 1, 0, 1)

#设置索引
df_new["index"] = range(len(df_new))
df_new= df_new.set_index("index")

In [18]:
pred_coords={"obs_id":range(0,136)}
with log_model1:
    pm.set_data({"anxiety": df_new["anxiety_r"],
                 "avoidance":df_new["avoidance_r"],
                 "y":df_new["romantic"]},
                coords=pred_coords)

    prediction = pm.sample_posterior_predictive(log_model1_trace, 
                                                var_names=["y_est"],
                                                predictions=True,
                                                extend_inferencedata=True,
                                                random_seed=84735)

Sampling: [y_est]


In [19]:
# 提取储存在 predicitons中的预测值
y_pred = prediction.predictions["y_est"].stack(sample=("chain","draw","obs_id")).values
# 统计其中0和1的个数，并除以总数，得到0和1对应的比例值
y_pred_freq = np.bincount(y_pred)/len(y_pred)
#绘制柱状图
bars = plt.bar([0, 1], y_pred_freq, color="#70AD47")
#用于在柱状图上标明比例值
for bar, freq in zip(bars, y_pred_freq):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{freq:.2f}", ha='center', va='bottom')
#对刻度、标题、坐标轴标题进行设置
plt.xticks([0, 1])
plt.suptitle("Out-of-sample prediction(X=1)")
plt.xlabel("romantic")
plt.ylabel("proportion")
sns.despine()

<Figure size 640x480 with 1 Axes>

In [20]:
#将原数据中的X 和Y存入数据框
pred_pi["anxiety"] = log_model1_ppc.constant_data.anxiety.values
pred_pi["avoidance"] = log_model1_ppc.constant_data.avoidance.values
pred_pi["romantic"] = log_model1_ppc.observed_data.y_est.values

#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,anxiety,avoidance,romantic,romantic_pred
obs_id,,,,,
0,0.79735,-1.975260,0.781027,1,1
1,0.57810,-0.323469,1.681138,1,1
2,0.69795,-1.172962,0.893541,1,1
3,0.43165,0.384442,0.218458,1,0
4,0.45320,0.148472,-1.244223,0,0
...,...,...,...,...,...
132,0.42590,0.337248,-0.737910,0,0
133,0.54525,-0.370663,-0.681653,1,1
134,0.75775,-1.644902,0.612256,1,1


In [21]:
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,45,24
1,26,42


In [22]:
# 计算a b c d的数量
true_positive =confusion_matrix.at[0, 1]
false_positive = confusion_matrix.at[1, 1]
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
# 代入公式
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("准确性:", accuracy)
print("敏感性:", sensitivity)
print("特异性:", specificity)

准确性: 0.36496350364963503
敏感性: 0.48
特异性: 0.5172413793103449


### 5. 与课上的模型(log_model1:自变量为回避倾向; log_model2:自变量为性别)进行模型比较  

In [23]:
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')

df = df_raw[df_raw["Site"] == "Oxford"]

df = df[["romantic", "anxiety_r", "avoidance_r","sex"]]

df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)

df["sex"] =  np.where(df['sex'] == 1, 0, 1)

df["index"] = range(len(df))
df = df.set_index("index")

In [24]:
with pm.Model() as log_model2:
    # 此处对coords的定义方式进行了更改，因为后续我们需要进行对新数据的预测
    # 因此将维度定义成可更改的
    log_model2.add_coord('obs_id',df.index, mutable=True)
    anxiety = pm.MutableData("anxiety", df.anxiety_r, dims="obs_id")
    avoidance = pm.MutableData("avoidance", df.avoidance_r, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * anxiety + beta_2 * avoidance, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")
with log_model2:
    # MCMC 近似后验分布
    log_model2_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)


with pm.Model() as log_model3:
    # 此处对coords的定义方式进行了更改，因为后续我们需要进行对新数据的预测
    # 因此将维度定义成可更改的
    log_model3.add_coord('obs_id',df.index, mutable=True)
    avoidance = pm.MutableData("avoidance", df.avoidance_r, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * avoidance, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")
with log_model3:
    # MCMC 近似后验分布
    log_model3_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,
                                idata_kwargs={"log_likelihood": True},  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)


coords = {"obs_id": df.index}

with pm.Model() as log_model4:
    log_model4.add_coord('obs_id',df.index, mutable=True)
    sex= pm.MutableData("sex", df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_2 *sex, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

with log_model4:
    # MCMC 近似后验分布
    log_model4_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True, 
                                idata_kwargs={"log_likelihood": True}, # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 9 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 8 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 9 seconds.


In [25]:
log_likelihood = pm.compute_log_likelihood(log_model2_trace,model=log_model2)

In [26]:
comparison_list = {
    "model1(anxiety、avoidance)":log_model2_trace,
    "model2(avoidance)":log_model3_trace,
    "model3(sex)":log_model4_trace,
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model1(anxiety、avoidance),0,-88.929794,2.671020,0.000000,1.000000e+00,3.586976,0.000000,False,log
model3(sex),1,-96.495929,1.568530,7.566136,8.604228e-15,0.176821,3.573746,False,log
model2(avoidance),2,-96.622943,1.834341,7.693150,0.000000e+00,0.551328,3.555412,False,log


### model1的elpd_LOO更高，预测性更好，model3最差